### Converting windows to 1D vectors and then 2D arrays

1. since we need to give CNN a matrix input of some sort, we will first convert each nucleotide to a 1D vector 
2. then each window will be a 2D array of vectors  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json 

from sklearn.model_selection import train_test_split

In [143]:
ago1234 = pd.read_csv('../data/intermediate/ago1234_padded.csv', index_col=0)
ago1234.head()

,ID,sequence_identity,sequence,RBP_binding,seq_len,padded_seqs,padded_len
0,"CID_041533;chrX,47028785,47028819,+",positive_test,gaacacaucaccugggccucuugcaccuuuuagaaagggcaaccuu...,1,334,gaacacaucaccugggccucuugcaccuuuuagaaagggcaaccuu...,400
1,"CID_014973;chr15,90774588,90774613,-",positive_test,guccuguggcguuuguucuccuaggccaaccccuucaaggagcgaa...,1,325,guccuguggcguuuguucuccuaggccaaccccuucaaggagcgaa...,400
2,"CID_015737;chr16,30959327,30959347,+",positive_test,ccuucccccugacccugacuccuugaacgucacugaaaacggcagc...,1,320,ccuucccccugacccugacuccuugaacgucacugaaaacggcagc...,400
3,"CID_025246;chr2,227661131,227661173,-",positive_test,ugccucaccccaaacccccaguggagagcagcggugguaagcucuu...,1,342,ugccucaccccaaacccccaguggagagcagcggugguaagcucuu...,400
4,"CID_009003;chr12,6347000,6347052,+",positive_test,gacccaugucucucccuuucccucagccuuccuucagaucaaacca...,1,352,gacccaugucucucccuuucccucagccuuccuucagaucaaacca...,400


In [144]:
## we append three rows at the top and bottom of each sequence with [0.25, 0.25, 0.25, 0.25] 
#so that the neural network does not get confused in thinking that this is an important pattern

def get_RNA_conv_array(seq, buffer=3):
    seq_len = len(seq)
    #print(seq_len)
    
    alpha='acgu'
    arr_len = seq_len + (buffer*2)
    #print(arr_len)
    seq_arr = np.zeros([arr_len, 4])
    
    for idx in range(buffer):
        #print(idx)
        seq_arr[idx] = np.array([0.25]*4)
        seq_arr[-(idx+1)] = np.array([0.25]*4)
        #print(seq_arr)
    
    for idx,char in enumerate(seq):
        idx = idx+3
        if char not in alpha:
            seq_arr[idx] = np.array([0.25]*4)
        else:
            index = alpha.index(char)
            seq_arr[idx][index] = 1
            
    return seq_arr

### Model 1

First we will train a model using only the sequence data, then we will train another using both the sequence data and the base-pairing data and compare performance. 

The authors who originally created this dataset included an external testing dataset that we can use for additional validation. Therefore, I will not include this dataset in any of the training and will use it for additional validation at the end (this dataset includes 459 sequences that do not bind the RBP and 448 that do bind). 

In [145]:
ago1234.head()

,ID,sequence_identity,sequence,RBP_binding,seq_len,padded_seqs,padded_len
0,"CID_041533;chrX,47028785,47028819,+",positive_test,gaacacaucaccugggccucuugcaccuuuuagaaagggcaaccuu...,1,334,gaacacaucaccugggccucuugcaccuuuuagaaagggcaaccuu...,400
1,"CID_014973;chr15,90774588,90774613,-",positive_test,guccuguggcguuuguucuccuaggccaaccccuucaaggagcgaa...,1,325,guccuguggcguuuguucuccuaggccaaccccuucaaggagcgaa...,400
2,"CID_015737;chr16,30959327,30959347,+",positive_test,ccuucccccugacccugacuccuugaacgucacugaaaacggcagc...,1,320,ccuucccccugacccugacuccuugaacgucacugaaaacggcagc...,400
3,"CID_025246;chr2,227661131,227661173,-",positive_test,ugccucaccccaaacccccaguggagagcagcggugguaagcucuu...,1,342,ugccucaccccaaacccccaguggagagcagcggugguaagcucuu...,400
4,"CID_009003;chr12,6347000,6347052,+",positive_test,gacccaugucucucccuuucccucagccuuccuucagaucaaacca...,1,352,gacccaugucucucccuuucccucagccuuccuucagaucaaacca...,400


In [146]:
external_validation = []
for index,row in ago1234.iterrows():
    identity = row['sequence_identity'].split('_')
    if identity[1] == 'test':
        external_validation.append(1)
    else:
        external_validation.append(0)
ago1234['external_validation'] = external_validation

In [147]:
ago1234.head()

,ID,sequence_identity,sequence,RBP_binding,seq_len,padded_seqs,padded_len,external_validation
0,"CID_041533;chrX,47028785,47028819,+",positive_test,gaacacaucaccugggccucuugcaccuuuuagaaagggcaaccuu...,1,334,gaacacaucaccugggccucuugcaccuuuuagaaagggcaaccuu...,400,1
1,"CID_014973;chr15,90774588,90774613,-",positive_test,guccuguggcguuuguucuccuaggccaaccccuucaaggagcgaa...,1,325,guccuguggcguuuguucuccuaggccaaccccuucaaggagcgaa...,400,1
2,"CID_015737;chr16,30959327,30959347,+",positive_test,ccuucccccugacccugacuccuugaacgucacugaaaacggcagc...,1,320,ccuucccccugacccugacuccuugaacgucacugaaaacggcagc...,400,1
3,"CID_025246;chr2,227661131,227661173,-",positive_test,ugccucaccccaaacccccaguggagagcagcggugguaagcucuu...,1,342,ugccucaccccaaacccccaguggagagcagcggugguaagcucuu...,400,1
4,"CID_009003;chr12,6347000,6347052,+",positive_test,gacccaugucucucccuuucccucagccuuccuucagaucaaacca...,1,352,gacccaugucucucccuuucccucagccuuccuucagaucaaacca...,400,1


In [148]:
ago1234_train = ago1234.loc[ago1234['external_validation'] == 0]
external_val = ago1234.loc[ago1234['external_validation'] == 1]

In [149]:
ago1234 = ago1234_train
ago1234.head()

,ID,sequence_identity,sequence,RBP_binding,seq_len,padded_seqs,padded_len,external_validation
139424,"PARCLIP_AGO1234.train.neg_26225;chr6,100855730...",negative_train,ucuugauaaugauguuucagacaugacaccaaaagcacugacaaca...,0,354,ucuugauaaugauguuucagacaugacaccaaaagcacugacaaca...,400,0
139425,"PARCLIP_AGO1234.train.neg_15504;chr19,37132064...",negative_train,gccaggcuggucuccaaugccugaccucaagugaucuacccgccua...,0,319,gccaggcuggucuccaaugccugaccucaagugaucuacccgccua...,400,0
139426,"PARCLIP_AGO1234.train.neg_2101;chr10,70137203,...",negative_train,aggaaagaugaaaaucacaauguuuuagcuaauuuuuaauauuaaa...,0,364,aggaaagaugaaaaucacaauguuuuagcuaauuuuuaauauuaaa...,400,0
139427,"PARCLIP_AGO1234.train.neg_16713;chr2,103349661...",negative_train,acucugagcucaaggaaaauacauauuucaugcgaguucuuucuuu...,0,326,acucugagcucaaggaaaauacauauuucaugcgaguucuuucuuu...,400,0
139428,"PARCLIP_AGO1234.train.neg_9764;chr16,4701858,4...",negative_train,cagcgugcaggggucuggcguugaggcugccuggcuggauuugcuc...,0,328,cagcgugcaggggucuggcguugaggcugccuggcuggauuugcuc...,400,0


In [150]:
ago1234.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62211 entries, 139424 to 276846
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   62211 non-null  object
 1   sequence_identity    62211 non-null  object
 2   sequence             62211 non-null  object
 3   RBP_binding          62211 non-null  int64 
 4   seq_len              62211 non-null  int64 
 5   padded_seqs          62211 non-null  object
 6   padded_len           62211 non-null  int64 
 7   external_validation  62211 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 4.3+ MB


In [151]:
ago1234.RBP_binding.value_counts()

RBP_binding
1    33112
0    29099
Name: count, dtype: int64

In [152]:
external_val.head()

,ID,sequence_identity,sequence,RBP_binding,seq_len,padded_seqs,padded_len,external_validation
0,"CID_041533;chrX,47028785,47028819,+",positive_test,gaacacaucaccugggccucuugcaccuuuuagaaagggcaaccuu...,1,334,gaacacaucaccugggccucuugcaccuuuuagaaagggcaaccuu...,400,1
1,"CID_014973;chr15,90774588,90774613,-",positive_test,guccuguggcguuuguucuccuaggccaaccccuucaaggagcgaa...,1,325,guccuguggcguuuguucuccuaggccaaccccuucaaggagcgaa...,400,1
2,"CID_015737;chr16,30959327,30959347,+",positive_test,ccuucccccugacccugacuccuugaacgucacugaaaacggcagc...,1,320,ccuucccccugacccugacuccuugaacgucacugaaaacggcagc...,400,1
3,"CID_025246;chr2,227661131,227661173,-",positive_test,ugccucaccccaaacccccaguggagagcagcggugguaagcucuu...,1,342,ugccucaccccaaacccccaguggagagcagcggugguaagcucuu...,400,1
4,"CID_009003;chr12,6347000,6347052,+",positive_test,gacccaugucucucccuuucccucagccuuccuucagaucaaacca...,1,352,gacccaugucucucccuuucccucagccuuccuucagaucaaacca...,400,1


In [153]:
external_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 907 entries, 0 to 70211
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   907 non-null    object
 1   sequence_identity    907 non-null    object
 2   sequence             907 non-null    object
 3   RBP_binding          907 non-null    int64 
 4   seq_len              907 non-null    int64 
 5   padded_seqs          907 non-null    object
 6   padded_len           907 non-null    int64 
 7   external_validation  907 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 63.8+ KB


In [154]:
external_val.RBP_binding.value_counts()

RBP_binding
0    459
1    448
Name: count, dtype: int64

In [155]:
X = []
y = []

for index,row in ago1234.iterrows():
    seq_subt = []
    seq_arr = get_RNA_conv_array(row['padded_seqs'])
    seq_subt.append(seq_arr.T)
    X.append(np.array(seq_subt))
    y.append(row['RBP_binding'])

In [156]:
X = np.array(X)
y = np.array(y)

X = np.swapaxes(X, -1, 1)

print(X.shape)
print(y.shape)

(62211, 406, 4, 1)
(62211,)


In [157]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(55989, 406, 4, 1)
(6222, 406, 4, 1)
(55989,)
(6222,)


In [158]:
np.save('../data/modeling/X_train2', X_train2)
np.save('../data/modeling/X_test2', X_test2)
np.save('../data/modeling/y_train2', y_train2)
np.save('../data/modeling/y_test2', y_test2)

Now for the external validation datasets. 

In [159]:
X_extval = []
y_extval = []

for index,row in external_val.iterrows():
    seq_subt = []
    seq_arr = get_RNA_conv_array(row['padded_seqs'])
    seq_subt.append(seq_arr.T)
    X_extval.append(np.array(seq_subt))
    y_extval.append(row['RBP_binding'])
    
X_extval = np.array(X_extval)
y_extval = np.array(y_extval)

X_extval = np.swapaxes(X_extval, -1, 1)

print(X_extval.shape)
print(y_extval.shape)

(907, 406, 4, 1)
(907,)


In [160]:
np.save('../data/modeling/X_extval', X_extval)
np.save('../data/modeling/y_extval', y_extval)

### Model 2

Now we will add the secondary structure information. We will use the same schema for the arrays, except we will use [0,0,0,0] to indicate an 'n', and we will place the base pairing probability in the array in the place of a 1. 

In [161]:
f = open('../data/intermediate/eterna.json')
eterna = json.load(f)
f.close()

In [162]:
print(list(eterna.keys())[0])
print(len(eterna))
print(eterna['CID_035006;chr6,47254049,47254070,-'].keys())

CID_035006;chr6,47254049,47254070,-
63118
dict_keys(['sequence', 'p_net_base_pairing'])


Now we will preprocess them by converting them to nice arrays.

In [163]:
def get_eterna_RNA_conv_array(seq, bp_data, buffer=3):
    seq_len = len(seq)
    if len(bp_data) != seq_len:
        to_add = len(seq) - len(bp_data) 
        bp_data = bp_data + [0 for i in range(to_add)]
    #print(seq_len)
    
    alpha='acgu'
    arr_len = seq_len + (buffer*2)
    #print(arr_len)
    seq_arr = np.zeros([arr_len, 4])
    
    for idx in range(buffer):
        #print(idx)
        seq_arr[idx] = np.array([0]*4)
        seq_arr[-(idx+1)] = np.array([0]*4)
        #print(seq_arr)
    
    for idx,char in enumerate(seq):
        #idx = idx+3
        val = bp_data[idx]
        if char not in alpha:
            seq_arr[idx+3] = np.array([0]*4)
        else:
            index = alpha.index(char)
            seq_arr[idx+3][index] = val
            
    return seq_arr

In [164]:
X_bp = []
y_bp = []

for index,row in ago1234.iterrows():
    seq = row['padded_seqs']
    bp_data = eterna[row['ID']]['p_net_base_pairing']
    #print(seq)
    #print(bp_data)
    seq_subt = []
    seq_arr = get_eterna_RNA_conv_array(seq, bp_data)
    seq_subt.append(seq_arr.T)
    X_bp.append(np.array(seq_subt))
    y_bp.append(row['RBP_binding'])

In [165]:
X_bp = np.array(X_bp)
y_bp = np.array(y_bp)

X_bp = np.swapaxes(X_bp, -1, 1)

print(X_bp.shape)
print(y_bp.shape)

(62211, 406, 4, 1)
(62211,)


In [166]:
X_bp_train, X_bp_test, y_bp_train, y_bp_test = train_test_split(X_bp, y_bp, test_size=0.1, 
                                                                random_state=42, stratify=y_bp)

print(X_bp_train.shape)
print(X_bp_test.shape)
print(y_bp_train.shape)
print(y_bp_test.shape)

(55989, 406, 4, 1)
(6222, 406, 4, 1)
(55989,)
(6222,)


In [167]:
np.save('../data/modeling/X_bp_train', X_bp_train)
np.save('../data/modeling/X_bp_test', X_bp_test)
np.save('../data/modeling/y_bp_train', y_bp_train)
np.save('../data/modeling/y_bp_test', y_bp_test)

now for the validation sets

In [168]:
X_bp_extval = []
y_bp_extval = []

for index,row in external_val.iterrows():
    seq_subt = []
    bp_data = eterna[row['ID']]['p_net_base_pairing']
    seq = row['padded_seqs']
    seq_arr = get_eterna_RNA_conv_array(seq, bp_data)
    seq_subt.append(seq_arr.T)
    X_bp_extval.append(np.array(seq_subt))
    y_bp_extval.append(row['RBP_binding'])
    
X_bp_extval = np.array(X_bp_extval)
y_bp_extval = np.array(y_bp_extval)

X_bp_extval = np.swapaxes(X_bp_extval, -1, 1)

print(X_bp_extval.shape)
print(y_bp_extval.shape)

(907, 406, 4, 1)
(907,)


In [169]:
np.save('../data/modeling/X_bp_extval', X_bp_extval)
np.save('../data/modeling/y_bp_extval', y_bp_extval)

### Modeling Thoughts

We will perform modeling on Google Colab since I can purchase GPUs to use to speed up processing. 

from here: https://levelup.gitconnected.com/a-practical-guide-to-selecting-cnn-architectures-for-computer-vision-applications-4a07ef90234

### Architectures to Try 
1. VGGNet - suitable for fine-grained classification tasks
2. LeNet - suitable for small-scale classification tasks


Since computational resources are limited, we might be limited to less complex architectures to limit training time/memory usage. 